# このノートブックは？
1. LightGBM_ALL_1, LightGBM_ALL_2の結果を結合
2. LightGBM_NYSE, LightGBM_NASDAQ, LightGBM_AMEXの結果を結合
3. MLP_NYSE, MLP_NASDAQの結果を結合
4. ExtraTrees_NYSE, ExtraTrees_NASDAQの結果を結合
5. 1-4を加重平均し、submissionファイル作成

## Google Driveのマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/'My Drive'/'PROBSPACE'/'StockPricePrediction'

Mounted at /content/drive
/content/drive/My Drive/PROBSPACE/StockPricePrediction


## ライブラリのインストール、インポート

In [ ]:
import src.install_libraries
import category_encoders as ce
from src.common_functions import *

  Created wheel for ta: filename=ta-0.8.0-py3-none-any.whl size=28895 sha256=6e0ed5ae922af5b6af2c3fc7da557c9a589de2f48cfc9c33ee63eedc0dd59b4b
  Stored in directory: /root/.cache/pip/wheels/7e/da/86/65cba22446ae2ef148de2079907264ef27feecfb7f51a45e0d
Successfully built ta


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## データの読み込み

In [ ]:
submission_lgb_ALL_1 = pd.read_csv(f'{Config.intermediate_dir_name}/submission_lgb_ALL_1.csv', dtype={'y': object})
submission_lgb_ALL_1.y = submission_lgb_ALL_1.y.apply(float)
submission_lgb_ALL_2 = pd.read_csv(f'{Config.intermediate_dir_name}/submission_lgb_ALL_2.csv', dtype={'y': object})
submission_lgb_ALL_2.y = submission_lgb_ALL_2.y.apply(float)

submission_lgb_NYSE = pd.read_csv(f'{Config.intermediate_dir_name}/submission_lgb_NYSE.csv', dtype={'y': object})
submission_lgb_NYSE.y = submission_lgb_NYSE.y.apply(float)
submission_lgb_NASDAQ = pd.read_csv(f'{Config.intermediate_dir_name}/submission_lgb_NASDAQ.csv', dtype={'y': object})
submission_lgb_NASDAQ.y = submission_lgb_NASDAQ.y.apply(float)
submission_lgb_AMEX = pd.read_csv(f'{Config.intermediate_dir_name}/submission_lgb_AMEX.csv', dtype={'y': object})
submission_lgb_AMEX.y = submission_lgb_AMEX.y.apply(float)

submission_mlp_NYSE = pd.read_csv(f'{Config.intermediate_dir_name}/submission_mlp_NYSE.csv', dtype={'y': object})
submission_mlp_NYSE.y = submission_mlp_NYSE.y.apply(float)
submission_mlp_NASDAQ = pd.read_csv(f'{Config.intermediate_dir_name}/submission_mlp_NASDAQ.csv', dtype={'y': object})
submission_mlp_NASDAQ.y = submission_mlp_NASDAQ.y.apply(float)

submission_et_NYSE = pd.read_csv(f'{Config.intermediate_dir_name}/submission_et_NYSE.csv', dtype={'y': object})
submission_et_NYSE.y = submission_et_NYSE.y.apply(float)
submission_et_NASDAQ = pd.read_csv(f'{Config.intermediate_dir_name}/submission_et_NASDAQ.csv', dtype={'y': object})
submission_et_NASDAQ.y = submission_et_NASDAQ.y.apply(float)

company_df2 = pd.read_csv(f'{Config.initial_dir_name}/company_list2.csv.gzip', index_col=0, compression='gzip')

## 1. LightGBM_ALL_1, LightGBM_ALL_2の結果を結合

In [ ]:
submission_lgb_ALL_1 = submission_lgb_ALL_1.set_index('id').join(company_df2.set_index('id'))[['y', 'List1']]
submission_lgb_ALL_2 = submission_lgb_ALL_2.set_index('id').join(company_df2.set_index('id'))[['y', 'List1']]

submission_lgb_ALL = submission_lgb_ALL_1.copy()
submission_lgb_ALL.y = (submission_lgb_ALL_1.y + submission_lgb_ALL_2.y) / 2
submission_lgb_ALL.loc[submission_lgb_ALL.List1=='NYSE', 'y'] = submission_lgb_ALL_2.y
submission_lgb_ALL.loc[submission_lgb_ALL.List1=='AMEX', 'y'] = submission_lgb_ALL_1.y

## 2. LightGBM_NYSE, LightGBM_NASDAQ, LightGBM_AMEXの結果を結合

In [ ]:
submission_lgb_NYSE = submission_lgb_NYSE.set_index('id').join(company_df2.set_index('id'))[['y', 'List1']]
submission_lgb_NASDAQ = submission_lgb_NASDAQ.set_index('id').join(company_df2.set_index('id'))[['y', 'List1']]
submission_lgb_AMEX = submission_lgb_AMEX.set_index('id').join(company_df2.set_index('id'))[['y', 'List1']]

submission_lgb_Concat = submission_lgb_NYSE.copy()
submission_lgb_Concat.loc[submission_lgb_Concat.List1=='NASDAQ', 'y'] = submission_lgb_NASDAQ.y
submission_lgb_Concat.loc[submission_lgb_Concat.List1=='AMEX', 'y'] = submission_lgb_AMEX.y

## 3. MLP_NYSE, MLP_NASDAQの結果を結合

In [ ]:
submission_mlp_NYSE = submission_mlp_NYSE.set_index('id').join(company_df2.set_index('id'))[['y', 'List1']]
submission_mlp_NASDAQ = submission_mlp_NASDAQ.set_index('id').join(company_df2.set_index('id'))[['y', 'List1']]

submission_mlp_Concat = submission_mlp_NYSE.copy()

## 4. ExtraTrees_NYSE, ExtraTrees_NASDAQの結果を結合

In [ ]:
submission_et_NYSE = submission_et_NYSE.set_index('id').join(company_df2.set_index('id'))[['y', 'List1']]
submission_et_NASDAQ = submission_et_NASDAQ.set_index('id').join(company_df2.set_index('id'))[['y', 'List1']]

submission_et_Concat = submission_et_NYSE.copy()
submission_et_Concat.loc[submission_et_Concat.List1=='NASDAQ', 'y'] = submission_et_NASDAQ.y

## 5. 1-4を加重平均し、submissionファイル作成

In [ ]:
blend_rates = {
    'NYSE': [0.0, 1.0, 0.0, 0.0],
    'NASDAQ': [0.0, 0.9, 0.0, 0.1],
    'AMEX': [0.0, 0.7, 0.3, 0.0],
    }
Lists = ['NYSE', 'NASDAQ', 'AMEX']
submission_df = submission_lgb_ALL.copy()
for l in Lists:
  submission_df.loc[submission_df.List1==l, 'y'] = submission_lgb_ALL.y * blend_rates[l][0] + submission_lgb_Concat.y * blend_rates[l][1] + submission_mlp_Concat.y * blend_rates[l][2] + submission_et_Concat.y * blend_rates[l][3]

In [ ]:
submission_df.drop('List1', axis=1, inplace=True)
submission_df.to_csv(f'{Config.submission_dir_name}/submission.csv')